### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [7]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-credentials.json", scope
)
gc = gspread.authorize(credentials)

### read in data

In [15]:
df_us = pd.read_csv("data/raw/us_lowy.csv", skiprows=1)

In [16]:
df_china = pd.read_csv("data/raw/china_lowy.csv", skiprows=1)

In [19]:
df = pd.concat([df_china, df_us]).reset_index()

In [24]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [31]:
summary_donations = (
    df.groupby(["donor_name", "recipient_name"])
    .agg(total_spent=pd.NamedAgg(column="spent_v", aggfunc=sum))
    .reset_index()
    .pivot(index="recipient_name", columns="donor_name", values="total_spent")
    .reset_index()
)

#### based on these topline figures, this is the accurate dataset

In [33]:
summary_donations["China"].sum()

2112004474.1199996

In [34]:
summary_donations["United States"].sum()

1850885503.0

In [35]:
summary_donations

donor_name,recipient_name,China,United States
0,Cook Islands,6.315566e+07,72680.0
1,Fiji,3.478127e+08,24253128.0
2,Kiribati,2.734920e+07,70000.0
3,Marshall Islands,NaN,513206870.0
4,"Micronesia, Fed. Sts.",6.933541e+07,868669977.0
5,Niue,2.200000e+05,63382.0
6,"Oceania, regional",2.198200e+06,80034401.0
7,Palau,NaN,163390838.0
8,Papua New Guinea,7.598941e+08,73120596.0
9,Samoa,3.549376e+08,13492725.0


### time series and other countries

In [38]:
aus_df = pd.read_csv("data/raw/aus_lowy.csv", skiprows=1)
nz_df = pd.read_csv("data/raw/nz_lowy.csv", skiprows=1)
adb_df = pd.read_csv("data/raw/adb_lowy.csv", skiprows=1)

In [39]:
df = pd.concat([df_us, df_china, aus_df, nz_df, adb_df])

In [41]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [46]:
annual = (
    df.groupby(["end_date", "donor_name"])
    .agg(annual_spent=pd.NamedAgg(column="spent_v", aggfunc=sum))
    .reset_index()
    .rename(columns={"end_date": "year"})
)

In [51]:
annual = annual.pivot(
    index="year", columns="donor_name", values="annual_spent"
).reset_index()

In [52]:
annual = annual[(annual["year"] >= 2009) & (annual["year"] <= 2019)]

In [56]:
annual = annual.reset_index().drop(columns="index")

In [57]:
dw.add_data(chart_id="dB15v", data=annual)

<Response [204]>